In [1]:
# AutoEncoders

# Importing the libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [89]:
# # Importing the dataset
movies = pd.read_csv('s1_improvised_only_grades.csv')
# users = pd.read_csv('ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
# ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')


In [7]:
pwd

'/home/adish/Desktop'

In [13]:
# Preparing the training set and the test set
training_set = pd.read_csv('s1_train.csv')
training_set = np.array(training_set, dtype = 'int')#user_id,movie_id,ratings
test_set = pd.read_csv('s1_test.csv')
test_set = np.array(test_set, dtype = 'int')
test_set.shape
#training_set.shape

(11, 10)

In [82]:
test_set.shape
#training_set.shape

torch.Size([11, 10])

In [81]:
#nb_students = int(max(max(training_set[:,0]), max(test_set[:,0])))
#nb_courses = int(max(max(training_set[:,1]), max(test_set[:,1])))
nb_students = max(training_set.shape[0], test_set.shape[0])-1
nb_courses = max(training_set.shape[1], test_set.shape[1])
nb_students
#nb_courses

42

In [44]:
# # Converting the data into an array with users in lines and movies in columns
# def convert(data):
#     new_data = []
#     for id_users in range(1, nb_users + 1):
#         id_movies = data[:,1][data[:,0] == id_users]
#         id_ratings = data[:,2][data[:,0] == id_users]
#         ratings = np.zeros(nb_movies)
#         ratings[id_movies - 1] = id_ratings
#         new_data.append(list(ratings))
#     return new_data
# training_set = convert(training_set)
# test_set = convert(test_set)

# Converting the data into Torch tensors
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)


In [68]:
# Creating the architecture of the Neural Network
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_courses, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_courses)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x.view(x.size(0), -1) 
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)


In [56]:
# # Training the SAE
# nb_epoch = 200
# for epoch in range(1, nb_epoch + 1):
#     train_loss = 0
#     s = 0.
#     for id_user in range(nb_users):
#         input = Variable(training_set[id_user]).unsqueeze(0)
#         target = input.clone()
#         if torch.sum(target.data > 0) > 0:
#             output = sae(input)
#             target = Variable(test_set[id_user]).unsqueeze(0)
#             output[target == 0] = 0
#             loss = criterion(output, target)
#             mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
#             loss.backward()
#             train_loss += np.sqrt(loss.data*mean_corrector)
#             s += 1.
#             optimizer.step()
#     print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))


In [57]:
# # Testing the SAE
# test_loss = 0
# s = 0.
# for id_user in range(nb_users):
#     input = Variable(training_set[id_user]).unsqueeze(0)
#     #target = Variable(test_set[id_user])
#     target = Variable(test_set[id_user]).unsqueeze(0)
#     if torch.sum(target.data > 0) > 0:
#         output = sae(input)
#         target.require_grad = False
#         output[(target == 0).unsqueeze(0)] = 0
#         loss = criterion(output, target)
#         mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
#         test_loss += np.sqrt(loss.data*mean_corrector)
#         s += 1.
# print('test loss: '+str(test_loss/s))

In [69]:
# Training the SAE
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_students):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
    
        if torch.sum(target.data > 0) > 0:
            output = sae(input.unsqueeze_(2))
            print(input.shape)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_courses/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            #train_loss += np.sqrt(loss.data[0]*mean_corrector)
            train_loss += np.sqrt(loss.item()*mean_corrector)

            s += 1.
            optimizer.step()
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))
# data = data.view(data.size(0), -1)
# output = model(data)[]

RuntimeError: size mismatch, m1: [10 x 1], m2: [10 x 20] at /opt/conda/conda-bld/pytorch_1535493744281/work/aten/src/TH/generic/THTensorMath.cpp:2070

In [72]:
# Training the SAE
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_students):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_courses/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            
            train_loss += np.sqrt(loss.item()*mean_corrector)

            s += 1.
            optimizer.step()
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))


epoch: 1 loss: 5.822364061959979
epoch: 2 loss: 5.76038912344871
epoch: 3 loss: 5.923290105586815
epoch: 4 loss: 5.884427517667222
epoch: 5 loss: 5.908632584340508
epoch: 6 loss: 5.794961382869698
epoch: 7 loss: 5.849532125425715
epoch: 8 loss: 5.761682807342576
epoch: 9 loss: 5.824416393686232
epoch: 10 loss: 5.906709383159256
epoch: 11 loss: 5.959960050947148
epoch: 12 loss: 5.802504752833365
epoch: 13 loss: 5.886707644644789
epoch: 14 loss: 5.84699178381337
epoch: 15 loss: 5.932601694849039
epoch: 16 loss: 5.873939333181759
epoch: 17 loss: 5.857056818691206
epoch: 18 loss: 5.82089726322953
epoch: 19 loss: 5.8721791960974326
epoch: 20 loss: 5.722315295046353
epoch: 21 loss: 5.917035395922646
epoch: 22 loss: 5.921207345906071
epoch: 23 loss: 5.874157163239762
epoch: 24 loss: 5.736997853080184
epoch: 25 loss: 5.909270032259671
epoch: 26 loss: 5.853838810564307
epoch: 27 loss: 5.932291215487543
epoch: 28 loss: 5.840770978662389
epoch: 29 loss: 5.930198597113308
epoch: 30 loss: 5.8269416

In [84]:
# Testing the SAE
test_loss = 0
s = 0.
for id_user in range(nb_students):
    input = Variable(training_set[id_user]).unsqueeze(0)
    #target = Variable(test_set[id_user]).unsqueeze(0)
for id_user in range(test_set.shape[0]):
    target = Variable(test_set[id_user]).unsqueeze(0)


    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.require_grad = False
        output[target == 0] = 0
        loss = criterion(output, target)
        mean_corrector = nb_courses/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.item()*mean_corrector)

        s += 1.
print('test loss: '+str(test_loss/s))

test loss: 6.799864062208514


In [ ]:
for i in range(1,1682):
    print(target[0][i])

In [ ]:
target.shape

In [92]:
user_id = 0
movie_title = movies.iloc[:nb_courses, 1:2]
print(movie_title)
user_rating = training_set.data.numpy()[user_id, :].reshape(-1,1)
user_target = test_set.data.numpy()[user_id, :].reshape(-1,1)
 
user_input = Variable(training_set[user_id]).unsqueeze(0)
predicted = sae(user_input)
predicted = predicted.data.numpy().reshape(-1,1)
 
# Join all info in one dataset
result_array = np.hstack([movie_title, user_target, predicted])
result_array = result_array[result_array[:, 1] > 0]
result_df = pd.DataFrame(data=result_array, columns=['Movie', 'Target Rating', 'Predicted'])

   MA101
0    0.0
1    6.0
2   10.0
3    6.0
4   10.0
5    9.0
6    5.0
7    6.0
8    0.0
9    9.0


In [91]:
result_df

,Movie,Target Rating,Predicted
0,0.0,62.0,29.525795
1,6.0,5.0,6.248788
2,10.0,7.0,7.200478
3,9.0,6.0,8.887316
4,5.0,5.0,8.523261
5,6.0,7.0,8.704281
6,0.0,9.0,8.247253
7,9.0,8.0,9.006348


In [93]:
user_id = 0
movie_title = movies.iloc[:nb_courses, 1:2]
print(movie_title)
user_rating = training_set.data.numpy()[user_id, :].reshape(-1,1)
user_target = test_set.data.numpy()[user_id, :].reshape(-1,1)
 
user_input = Variable(training_set[user_id]).unsqueeze(0)
predicted = sae(user_input)
predicted = predicted.data.numpy().reshape(-1,1)
 
# Join all info in one dataset
result_array = np.hstack([movie_title, user_target, predicted])
result_array = result_array[result_array[:, 1] > 0]
result_df = pd.DataFrame(data=result_array, columns=['Movie', 'Target Rating', 'Predicted'])

   MA101
0    0.0
1    6.0
2   10.0
3    6.0
4   10.0
5    9.0
6    5.0
7    6.0
8    0.0
9    9.0
